In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import random
import functools
from functools import reduce

In [2]:
image_size = 28
input_num_units=28*28
num_labels = 10
output_num_units=num_labels
learning_rate=0.5
num_steps =501
num_train_samples = 10000

In [3]:
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)#Concise 1-Hot Encoding in NUMPY
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def getSampleIndexes(population,sample_sz):
    if(sample_sz>population.shape[0]):
        return np.array(range(population.shape[0]))
    rand_indexes=random.sample(range(population.shape[0]),sample_sz)
    return population[rand_indexes,:],rand_indexes
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])
def isEqual(nparr1,nparr2):
    totDims=len(nparr1.shape)
    if np.sum([nparr1.shape[i]==nparr2.shape[i] for i in range(totDims)])!=totDims:
        return False
    eqItems=nparr1==nparr2
    if np.sum(eqItems)==reduce(lambda x,y:x*y,nparr1.shape):
        return True
    else:
        return False

In [6]:
#Logistic Regression : Declaring the Model as Computational Graph
graph_reg = tf.Graph()
with graph_reg.as_default():
    train_dataset_tf = tf.placeholder(tf.float32, [None, input_num_units])
    train_labels_tf = tf.placeholder(tf.float32, [None, num_labels])
    valid_dataset_tf = tf.placeholder(tf.float32, [None, input_num_units])
    #valid_labels_tf = tf.placeholder(tf.float32, [None, num_labels])
    test_dataset_tf = tf.placeholder(tf.float32, [None, input_num_units])
    #test_labels_tf = tf.placeholder(tf.float32, [None, num_labels])    
    
    weights = tf.Variable(tf.truncated_normal([input_num_units, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    weighted_sum = tf.matmul(train_dataset_tf, weights) + biases 
    cost=tf.nn.softmax_cross_entropy_with_logits(labels=train_labels_tf, logits=weighted_sum)
    loss_fun = tf.reduce_mean(cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_fun)
    
    train_prediction = tf.nn.softmax(tf.matmul(train_dataset_tf, weights) + biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_dataset_tf, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_dataset_tf, weights) + biases)

In [7]:
def getFeedDict(train_x,train_y):
        return {train_dataset_tf:train_x,train_labels_tf:train_y,valid_dataset_tf:valid_dataset,
              test_dataset_tf:test_dataset}
def validate_tf_comput_graph(session,tf_feed_dict):
    _=session.run([optimizer],feed_dict=tf_feed_dict)
    train_pred_1=session.run(train_prediction,feed_dict=tf_feed_dict)
    train_pred_2=session.run(train_prediction,feed_dict=tf_feed_dict)
    if isEqual(train_pred_1,train_pred_2):
        print("Computational Graph : Validation Passed!")
    else:
        print("Computational Graph : Validation Failed!")
with tf.Session(graph=graph_reg) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    train_x,rand_indexes=getSampleIndexes(train_dataset,num_train_samples)
    train_y=train_labels[rand_indexes]
    validate_tf_comput_graph(session,getFeedDict(train_x,train_y))
    for step in range(num_steps):
        train_x,rand_indexes=getSampleIndexes(train_dataset,num_train_samples)
        train_y=train_labels[rand_indexes]
        _,loss,train_pred=session.run([optimizer,loss_fun, train_prediction],feed_dict=getFeedDict(train_x,train_y))
        if step%100==0:
            print('Loss at step %d: %f' % (step, loss))
            valid_pred,test_pred=session.run([valid_prediction,test_prediction],feed_dict=getFeedDict(train_x,train_y))
            train_acc=accuracy(train_pred, train_y)
            valid_acc=accuracy(valid_pred, valid_labels)
            test_acc=accuracy(test_pred,test_labels)
            print("Accuracy ~~~~~~~~~~~~>",train_acc,valid_acc,test_acc )
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

Initialized
Computational Graph : Validation Passed!
Loss at step 0: 13.153976
Accuracy ~~~~~~~~~~~~> 12.06 14.73 15.43
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 100: 2.360307
Accuracy ~~~~~~~~~~~~> 71.37 70.47 78.82
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 200: 1.990555
Accuracy ~~~~~~~~~~~~> 73.58 73.0 81.48
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 300: 1.797248
Accuracy ~~~~~~~~~~~~> 74.89 74.13 82.44
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 400: 1.644995
Accuracy ~~~~~~~~~~~~> 75.51 74.71 83.11
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Loss at step 500: 1.472335
Accuracy ~~~~~~~~~~~~> 76.73 75.11 83.47
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
